#### Churn Modeling (XGBoost)

In [1]:
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = pd.read_csv("C:/Users/user/Desktop/kaggle competetion/Churn_Modelling.csv")
dataset.head()
#dataset.describe()
#dataset.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
x = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]

In [4]:
# this code is deprecated in 0.22 version

'''from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder_x_1 =LabelEncoder()
x[:,1] = labelencoder_x_1.fit_transform(x[:,1])
labelencoder_x_2 = LabelEncoder()
x[:,2] = labelencoder_x_2.fit_transform(x[:,2])
onehotencoder = OneHotEncoder(categorical_features = [1])
x = onehotencoder.fit_transform(x).toarray()
x = x[:, 1:]'''


'from sklearn.preprocessing import LabelEncoder,OneHotEncoder\nlabelencoder_x_1 =LabelEncoder()\nx[:,1] = labelencoder_x_1.fit_transform(x[:,1])\nlabelencoder_x_2 = LabelEncoder()\nx[:,2] = labelencoder_x_2.fit_transform(x[:,2])\nonehotencoder = OneHotEncoder(categorical_features = [1])\nx = onehotencoder.fit_transform(x).toarray()\nx = x[:, 1:]'

In [5]:
num_cols = [col for col in x.columns if x[col].dtypes != "O"]
cat_cols = [col for col in x.columns if x[col].dtypes == "O" ]

In [6]:
from sklearn.preprocessing import OneHotEncoder,RobustScaler
from sklearn.compose import ColumnTransformer
ohe = OneHotEncoder(sparse = False,handle_unknown = "ignore")
ct = ColumnTransformer([
    ("step1",RobustScaler(),num_cols),
    ("step2",ohe,cat_cols)
], remainder = 'drop')
#ct1 = ColumnTransformer([("Gender", OneHotEncoder(), [1])], remainder = 'passthrough')
x = ct.fit_transform(x)
#x = ct1.fit_transform(x)
#x = x[:, 1:]


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

To remove warnings of xgboost using tpot:

In [8]:
from tpot import TPOTClassifier
from tpot.config import classifier_config_dict
classifier_config_dict['xgboost.XGBClassifier'] = {
    'n_estimators': [100],
    'max_depth': range(1, 11),
    'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
    'subsample': np.arange(0.05, 1.01, 0.05),
    'min_child_weight': range(1, 21),
    'n_jobs': [1], # replace "nthread"
    'verbosity': [0] # add this line to slient warning message
}
        
# for a quick test
tpot = TPOTClassifier(generations=2, population_size=10, verbosity=2,
                      config_dict=classifier_config_dict)
tpot.fit(x_train, y_train)


Generation 1 - Current best internal CV score: 0.8622499999999998

Generation 2 - Current best internal CV score: 0.8622499999999998

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.01, max_depth=8, min_child_weight=18, n_estimators=100, n_jobs=1, subsample=0.9500000000000001, verbosity=0)


TPOTClassifier(config_dict={'sklearn.cluster.FeatureAgglomeration': {'affinity': ['euclidean',
                                                                                  'l1',
                                                                                  'l2',
                                                                                  'manhattan',
                                                                                  'cosine'],
                                                                     'linkage': ['ward',
                                                                                 'complete',
                                                                                 'average']},
                            'sklearn.decomposition.FastICA': {'tol': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])},
                            'sklearn.decomposition.PC

In [9]:
from xgboost import XGBClassifier
xgbc = XGBClassifier()
print(xgbc)
xgbc.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, use_label_encoder=True,
              validate_parameters=None, verbosity=None)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [10]:
score = xgbc.score(x_train,y_train)
print("training score: ",score)

training score:  0.9515


In [11]:
from sklearn.model_selection import cross_val_score
cv_score = cross_val_score(xgbc,x_train,y_train,cv=10)

print("cv_mean_score: ",cv_score.mean())

cv_mean_score:  0.8554999999999999


In [12]:
y_pred = xgbc.predict(x_test)

In [15]:
# making the confussion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[1494,   83],
       [ 216,  207]], dtype=int64)

In [29]:
cm[1:,1:]

array([[207]], dtype=int64)

In [28]:
sum(cm[0:1,0:1],cm[1:,1:])

array([[1701]], dtype=int64)

In [41]:
a = sum(cm[0:1,0:1],cm[1:,1:])/sum(sum(cm))
float(a)

0.8505

##### For regression problem:

from xgboost import XGBRegressor

xgb_model = xgb.XGBRegressor(base_score=0.5,objective="reg:linear", random_state=42)

xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

mse=mean_squared_error(y, y_pred)

print(np.sqrt(mse))
